In [4]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import json

In [5]:
Directory = "UD_English-EWT/"
#Order: dev, test, train
Filename = ["en_ewt-ud-dev.conllu"]
Jsons = []

for f in Filename:
    Path = "".join((Directory, f))
    
    with open(Path, 'r', encoding='UTF-8') as j:
        i = 0
        for line in j:
            conllu = line.split()
   
    j.close()


In [1]:
def hasalpha(word):
    for x in word:
        if x.isalpha():
            return True
    return False

def hasnumeric(word):
    for x in word:
        if x.isnumeric():
            return True
    return False

def hasspecial(word):
    for x in word:
        if x.isalnum() is False:
            return True
    return False

def create_features(filepath):
    
    
    with open(filepath, 'r', encoding='UTF-8') as f:
        x_toadd = []
        y_toadd = []
        x_final = []
        y_final = []
        
        for line in f:
                       
            conllu = line.split()
            #case: end of sentence reached
            if len(conllu) == 0:
                x_final.append(x_toadd)
                y_final.append(y_toadd)
                x_toadd = []
                y_toadd = []                
            elif conllu[0].isnumeric():
                word = conllu[1]
                XPOS = conllu[4]
                extra_tags = conllu[5].split('|')
                wordtype = conllu[3]
                
                
                features = ['bias',
                            'word.lower=' + word.lower(),
                            'word.XPOS=' + XPOS,
                            'word.iscapitalized=%s' % word.istitle(),
                            'word.hasnumeric=%s' % hasnumeric(word),
                            'word.hasalpha=%s' % hasalpha(word),
                            'word.hasspecial=%s' % hasspecial(word)
                           ]
#                             'word.prefix=' + word[:3],
#                             'word.suffix=' + word[-3:]]
                
                if conllu[1].lower() != conllu[2].lower():
                    features.append('word.baseform=' + conllu[2].lower())
    
                for x in extra_tags:
                    if 'PronType' in x:
                        features.append('word.prontype=' + x[9:])
                    if 'Foreign' in x:
                        features.append('word.foreign=' + x[8:])
                        
                x_toadd.append(features)
                y_toadd.append(wordtype)
                
    f.close()
                
    return x_final, y_final
                

In [2]:
dev_x_unneighbored, dev_y = create_features("UD_English-EWT/en_ewt-ud-dev.conllu")
test_x_unneighbored, test_y = create_features("UD_English-EWT/en_ewt-ud-test.conllu")
train_x_unneighbored, train_y = create_features("UD_English-EWT/en_ewt-ud-train.conllu")

In [3]:
def add_neighbors(x_data):
    
    x_data_final = []
    
    for sentence in x_data:
        final_sentence = []
        for i in range(0, len(sentence)):
            modified_word = []
            modified_word.extend(sentence[i])

            if i == 0:
                modified_word.append('BOS')
            else:
                prev_word = sentence[i-1]
                modified_word.extend(['-1:' + x for x in prev_word[1:]])

            if i == (len(sentence)-1):
                modified_word.append('EOS')
            else:
                next_word = sentence[i+1]
                modified_word.extend(['+1:' + x for x in next_word[1:]])

            final_sentence.append(modified_word)
        x_data_final.append(final_sentence)
            
    return x_data_final

In [4]:
dev_x = add_neighbors(dev_x_unneighbored)
test_x = add_neighbors(test_x_unneighbored)
train_x = add_neighbors(train_x_unneighbored)

In [5]:
def x_test():
    for i in range(0, len(dev_y)):
        for j in range(0,len(dev_y[i])):
            if dev_y[i][j] == 'X':
                print(dev_x[i][j])
                return
            
x_test()

['bias', 'word.lower=a', 'word.XPOS=FW', 'word.iscapitalized=True', 'word.hasnumeric=False', 'word.hasalpha=True', 'word.hasspecial=False', 'word.foreign=Yes', 'BOS', '+1:word.lower=la', '+1:word.XPOS=FW', '+1:word.iscapitalized=False', '+1:word.hasnumeric=False', '+1:word.hasalpha=True', '+1:word.hasspecial=False', '+1:word.foreign=Yes']


In [59]:
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(train_x, train_y):
    trainer.append(xseq, yseq)

In [60]:
trainer.set_params({
    'c1': 1.0,   
    'c2': 1e-3,  
    'max_iterations': 50,  

    #'feature.possible_transitions': True
})

In [61]:
trainer.train('test')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 116685
Seconds required: 0.776

L-BFGS optimization
c1: 1.000000
c2: 0.001000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 487560.679278
Feature norm: 1.000000
Error norm: 101175.363612
Active features: 48802
Line search trials: 1
Line search step: 0.000007
Seconds required for this iteration: 0.814

***** Iteration #2 *****
Loss: 427577.567495
Feature norm: 1.809697
Error norm: 77635.720556
Active features: 50114
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.379

***** Iteration #3 *****
Loss: 326203.949570
Feature norm: 4.027720
Error norm: 84070.579276
Active features: 50385
Line search trials: 1
Line search step: 1.000000
Seconds required

***** Iteration #43 *****
Loss: 5524.923598
Feature norm: 76.470463
Error norm: 175.551160
Active features: 2436
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.411

***** Iteration #44 *****
Loss: 5509.426929
Feature norm: 77.291309
Error norm: 665.911180
Active features: 2301
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.460

***** Iteration #45 *****
Loss: 5480.859708
Feature norm: 77.645257
Error norm: 143.951354
Active features: 2274
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.467

***** Iteration #46 *****
Loss: 5461.333291
Feature norm: 77.951276
Error norm: 74.738239
Active features: 2234
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.464

***** Iteration #47 *****
Loss: 5443.486651
Feature norm: 78.893377
Error norm: 544.804008
Active features: 2148
Line search trials: 1
Line search step: 1.000000
Seconds require

In [62]:
tagger = pycrfsuite.Tagger()
tagger.open('test')

In [63]:
example_sent = test_x[1]

print("Predicted:", ' '.join(tagger.tag(example_sent)))
print("Correct:  ", ' '.join(test_y[1]))

Predicted: PRON SCONJ PROPN VERB ADP PRON NOUN PUNCT NOUN PUNCT CCONJ ADV NOUN PUNCT NOUN ADP DET ADV PUNCT ADJ NOUN NOUN PUNCT
Correct:   PRON SCONJ PROPN VERB ADP PRON NOUN PUNCT NOUN PUNCT CCONJ ADV NOUN PUNCT NOUN ADP DET ADV PUNCT ADJ NOUN NOUN PUNCT


In [49]:
def evaluate(data_x, data_y):
    scores = {}
    tagger_2 = pycrfsuite.Tagger()
    tagger_2.open('test')
    
    for i in range(0, len(data_x)):
        predicted = tagger_2.tag(data_x[i])
        actual = data_y[i]
        
        for j in range(0, len(predicted)):
            
            if predicted[j] not in scores:
                scores[predicted[j]] = {'TP': 0, 'FP': 0, 'FN': 0}
            if actual[j] not in scores:
                scores[actual[j]] = {'TP': 0, 'FP': 0, 'FN': 0}
                
            if predicted[j] == actual[j]:
                scores[predicted[j]]['TP'] += 1
            else:
                scores[predicted[j]]['FP'] += 1
                scores[actual[j]]['FN'] += 1
        
    return scores
        
def print_evaluate(data):
    for x in data.keys():
        metrics = data[x]
        precision = metrics['TP']/(metrics['TP'] + metrics['FP'])
        recall = metrics['TP']/(metrics['TP'] + metrics['FN'])
        F1 = 2*(precision*recall)/(precision + recall)
        
        print("Stats for %s:" % x)
        print("Precision: ", precision)
        print("Recall: ", recall)
        print("F1: ", F1)
        

In [64]:
print("Now evaluating dev data: ")
print()
results = evaluate(dev_x, dev_y)
print_evaluate(results)

Now evaluating dev data: 

Stats for ADP:
Precision:  0.9762481822588464
Recall:  0.9965363681345868
F1:  0.9862879529872673
Stats for DET:
Precision:  0.9984168865435357
Recall:  0.9989440337909187
F1:  0.9986803906043811
Stats for PROPN:
Precision:  1.0
Recall:  1.0
F1:  1.0
Stats for VERB:
Precision:  0.990149580445093
Recall:  0.9822656532754253
F1:  0.9861918604651163
Stats for NOUN:
Precision:  0.9995235826584088
Recall:  0.9997617345723135
F1:  0.999642644431209
Stats for PUNCT:
Precision:  0.9970883209317373
Recall:  0.9996756406097956
F1:  0.9983803045027535
Stats for NUM:
Precision:  1.0
Recall:  1.0
F1:  1.0
Stats for PART:
Precision:  0.9920886075949367
Recall:  0.9952380952380953
F1:  0.993660855784469
Stats for ADJ:
Precision:  1.0
Recall:  0.9994407158836689
F1:  0.9997202797202798
Stats for ADV:
Precision:  0.9968228752978554
Recall:  0.9913112164296999
F1:  0.9940594059405939
Stats for AUX:
Precision:  0.9686888454011742
Recall:  0.9834437086092715
F1:  0.9760105159382

In [65]:
print("Now evaluating test data: ")
print()
results = evaluate(test_x, test_y)
print_evaluate(results)

Now evaluating test data: 

Stats for PRON:
Precision:  0.9986065954482118
Recall:  0.9962928637627433
F1:  0.9974483878450475
Stats for SCONJ:
Precision:  0.9629629629629629
Recall:  0.8756476683937824
F1:  0.9172320217096337
Stats for PROPN:
Precision:  1.0
Recall:  1.0
F1:  1.0
Stats for VERB:
Precision:  0.9935238095238095
Recall:  0.983779705771407
F1:  0.9886277482941622
Stats for ADP:
Precision:  0.9761557177615572
Recall:  0.9935611688954928
F1:  0.9847815414825725
Stats for PUNCT:
Precision:  0.9974301317057501
Recall:  0.9996780424983902
F1:  0.9985528219971057
Stats for NOUN:
Precision:  0.999274661508704
Recall:  1.0
F1:  0.9996371991776515
Stats for CCONJ:
Precision:  1.0
Recall:  1.0
F1:  1.0
Stats for ADV:
Precision:  0.9926350245499181
Recall:  0.9893964110929854
F1:  0.9910130718954248
Stats for DET:
Precision:  0.9957916885849553
Recall:  0.9984177215189873
F1:  0.9971029760337109
Stats for ADJ:
Precision:  0.9994089834515366
Recall:  0.9994089834515366
F1:  0.9994089